In [1]:
import pandas as pd

from env import get_url

pd.set_option('display.max_columns', None)

# Data Wrangling

## Zillow Data

### Acquire

#### Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

* #### Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.

* #### Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.

* #### Only include properties that include a latitude and longitude value.

In [2]:
query = '''
SELECT prop.*, 
       pred.logerror, 
       pred.transactiondate, 
       air.airconditioningdesc, 
       arch.architecturalstyledesc, 
       build.buildingclassdesc, 
       heat.heatingorsystemdesc, 
       landuse.propertylandusedesc, 
       story.storydesc, 
       construct.typeconstructiondesc 
FROM   properties_2017 prop 
       INNER JOIN (SELECT parcelid, 
                          logerror, 
                          Max(transactiondate) transactiondate 
                   FROM   predictions_2017 
                   GROUP  BY parcelid, 
                             logerror) pred USING (parcelid) 
       LEFT JOIN airconditioningtype air USING (airconditioningtypeid) 
       LEFT JOIN architecturalstyletype arch USING (architecturalstyletypeid) 
       LEFT JOIN buildingclasstype build USING (buildingclasstypeid) 
       LEFT JOIN heatingorsystemtype heat USING (heatingorsystemtypeid) 
       LEFT JOIN propertylandusetype landuse USING (propertylandusetypeid) 
       LEFT JOIN storytype story USING (storytypeid) 
       LEFT JOIN typeconstructiontype construct USING (typeconstructiontypeid) 
WHERE  prop.latitude IS NOT NULL 
       AND prop.longitude IS NOT NULL; 
'''

url = get_url('zillow')

zillow = pd.read_sql(query, url)
zillow

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1087254,10711855,NaN,NaN,NaN,2.0,3.0,NaN,8.0,2.0,NaN,NaN,2107.0,2107.0,NaN,NaN,NaN,NaN,6037.0,NaN,2.0,NaN,NaN,NaN,2.0,34222559.0,-118617387.0,9158.0,1.0,NaN,NaN,NaN,1.0,0101,261.0,LARE9,6.037113e+07,12447.0,3101.0,268588.0,96339.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1972.0,NaN,NaN,249655.0,624139.0,2016.0,374484.0,7659.36,None,NaN,6.037113e+13,-0.007357,2017-07-07,None,None,None,Central,Single Family Residential,None,None
1,1072280,10711877,1.0,NaN,NaN,2.0,4.0,NaN,8.0,2.0,NaN,NaN,1882.0,1882.0,NaN,NaN,NaN,NaN,6037.0,NaN,2.0,NaN,NaN,NaN,2.0,34220261.0,-118616409.0,9035.0,1.0,NaN,NaN,NaN,1.0,0101,261.0,LARE9,6.037113e+07,12447.0,3101.0,268588.0,96339.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1972.0,NaN,NaN,253000.0,660000.0,2016.0,407000.0,8123.91,None,NaN,6.037113e+13,0.021066,2017-08-29,Central,None,None,Central,Single Family Residential,None,None
2,1340933,10711888,1.0,NaN,NaN,2.0,4.0,NaN,8.0,2.0,NaN,NaN,1882.0,1882.0,NaN,NaN,NaN,NaN,6037.0,NaN,2.0,NaN,NaN,NaN,2.0,34222491.0,-118616854.0,9800.0,NaN,NaN,NaN,NaN,NaN,0100,261.0,LARE9,6.037113e+07,12447.0,3101.0,268588.0,96339.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1972.0,NaN,NaN,257591.0,542923.0,2016.0,285332.0,6673.24,None,NaN,6.037113e+13,0.077174,2017-04-04,Central,None,None,Central,Single Family Residential,None,None
3,1878109,10711910,NaN,NaN,NaN,2.0,3.0,NaN,8.0,2.0,NaN,NaN,1477.0,1477.0,NaN,NaN,NaN,NaN,6037.0,NaN,2.0,NaN,NaN,NaN,2.0,34221864.0,-118615739.0,11285.0,1.0,NaN,NaN,NaN,1.0,0101,261.0,LARE11,6.037113e+07,12447.0,3101.0,268588.0,96339.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1960.0,NaN,NaN,57968.0,78031.0,2016.0,20063.0,1116.46,None,NaN,6.037113e+13,-0.041238,2017-03-17,None,None,None,Central,Single Family Residential,None,None
4,2190858,10711923,NaN,NaN,NaN,2.0,4.0,NaN,8.0,2.0,NaN,NaN,1918.0,1918.0,NaN,NaN,NaN,NaN,6037.0,NaN,2.0,NaN,NaN,NaN,2.0,34220619.0,-118615253.0,11239.0,1.0,NaN,NaN,NaN,1.0,0101,261.0,LARE11,6.037113e+07,12447.0,3101.0,268588.0,96339.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1960.0,NaN,NaN,167869.0,415459.0,2016.0,247590.0,5239.85,None,NaN,6.037113e+13,-0.009496,2017-03-24,None,None,None,Central,Single Family Residential,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77570,775695,167686999,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,NaN,34424104.0,-118468083.0,NaN,NaN,NaN,NaN,NaN,NaN,0100,261.0,SCRM,6.037920e+07,NaN,3101.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26405.0,2016.0,26405.0,988.48,None,NaN,NaN,-0.068632,2017-02-28,None,None,None,None,Single Family Residential,None,None
77571,2863262,167687739,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,NaN,

#### Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [3]:
zillow.describe()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror
count,7.757500e+04,7.757500e+04,25006.000000,206.000000,50.000000,77575.000000,77575.000000,15.000000,49809.000000,76960.000000,614.0,6035.000000,77375.000000,73920.000000,42.000000,3027.000000,6035.000000,386.000000,77575.000000,8287.000000,76960.000000,25517.000000,25517.000000,1539.0,49570.000000,7.757500e+04,7.757500e+04,6.931800e+04,16173.0,869.000000,465.0,1074.0,15078.0,77575.000000,7.757500e+04,76103.000000,77575.000000,30973.000000,77525.000000,77575.00000,50.0,10105.000000,222.000000,50703.000000,2393.000000,70.000000,77306.000000,17596.000000,172.0,7.746000e+04,7.757400e+04,77575.0,7.757300e+04,77570.000000,2900.000000,7.732800e+04,77575.000000
mean,1.495352e+06,1.300630e+07,1.812045,7.388350,679.720000,2.298518,3.053252,3.933333,6.533779,2.316385,66.0,1366.198012,1784.938998,1760.287297,1388.761905,2354.491245,1381.426678,2082.500000,6048.812568,1.191263,2.250104,1.815339,350.055845,1.0,3.921707,3.400837e+07,-1.182037e+08,2.997445e+04,1.0,517.930955,1.0,1.0,1.0,261.824467,6.049123e+07,33683.390392,2534.535933,187730.297162,96586.677033,1.47612,7.0,1.009599,6.040541,1.110309,305.460928,216.385714,1968.610936,1.434246,1.0,1.892815e+05,4.901447e+05,2016.0,3.011453e+05,5995.821528,14.088276,6.049666e+13,0.016802
std,8.609703e+05,3.478021e+06,2.965823,2.734542,689.703546,0.996700,1.140440,0.258199,1.722062,0.979684,0.0,670.803630,954.262540,934.364843,122.220874,1186.985442,725.904022,1240.382784,20.745309,0.491031,0.966479,0.588329,261.651602,0.0,3.594804,2.652878e+05,3.593826e+05,1.233076e+05,0.0,156.569664,0.0,0.0,0.0,5.141701,2.058768e+05,47212.856528,801.449852,165045.226146,3793.648751,2.82360,0.0,0.118578,0.557285,1.169967,238.735241,190.177514,23.793197,0.544518,0.0,2.304137e+05,6.538022e+05,0.0,4.927287e+05,7628.869090,2.181281,1.533376e+12,0.170743
min,3.490000e+02,1.071186e+07,1.000000,2.000000,38.000000,0.000000,0.000000,3.000000,1.000000,1.000000,66.0,44.000000,128.000000,128.000000,1056.000000,598.000000,44.000000,380.000000,6037.000000,1.000000,1.000000,0.000000,0.000000,1.0,1.000000,3.333953e+07,-1.194754e+08,2.360000e+02,1.0,24.000000,1.0,1.0,1.0,31.000000,6.037101e+07,3491.000000,1286.000000,6952.000000,95982.000000,0.00000,7.0,1.000000,4.000000,1.000000,11.000000,12.000000,1824.000000,1.000000,1.0,4.400000e+01,1.000000e+03,2016.0,1.610000e+02,19.920000,3.000000,6.037101e+13,-4.655420
25%,7.521050e+05,1.153820e+07,1.000000,7.000000,273.000000,2.000000,2.000000,4.000000,6.000000,2.000000,66.0,955.000000,1182.000000,1172.000000,1344.000000,1624.000000,956.000000,993.750000,6037.000000,1.000000,2.000000,2.000000,0.000000,1.0,2.000000,3.381463e+07,-1.184150e+08,5.700000e+03,1.0,424.000000,1.0,1.0,1.0,261.000000,6.037311e+07,12447.000000,1286.000000,46736.000000,96193.000000,0.00000,7.0,1.000000,6.000000,1.000000,170.000000,61.500000,1953.000000,1.000000,1.0,8.418200e+04,2.068985e+05,2016.0,8.529300e+04,2712.630000,14.000000,6.037311e+13,-0.024311
50%,1.498195e+06,1.253053e+07,1.000000,7.000000,515.000000,2.000000,3.000000,4.000000,6.000000,2.000000,66.0,1257.000000,1542.000000,1523.000000,1440.000000,2088.000000,1

In [4]:
zillow.describe(include='object')

,propertycountylandusecode,propertyzoningdesc,taxdelinquencyflag,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
count,77575,50475,2900,77575,25006,206,15,49570,77575,50,222
unique,75,1907,1,265,5,5,2,10,13,1,4
top,0100,LAR1,Y,2017-06-30,Central,Contemporary,Buildings having wood or wood and steel frames,Central,Single Family Residential,Basement,Frame
freq,26782,6766,2900,1194,23185,172,14,33633,52439,50,219


In [5]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77575 entries, 0 to 77574
Data columns (total 68 columns):
id                              77575 non-null int64
parcelid                        77575 non-null int64
airconditioningtypeid           25006 non-null float64
architecturalstyletypeid        206 non-null float64
basementsqft                    50 non-null float64
bathroomcnt                     77575 non-null float64
bedroomcnt                      77575 non-null float64
buildingclasstypeid             15 non-null float64
buildingqualitytypeid           49809 non-null float64
calculatedbathnbr               76960 non-null float64
decktypeid                      614 non-null float64
finishedfloor1squarefeet        6035 non-null float64
calculatedfinishedsquarefeet    77375 non-null float64
finishedsquarefeet12            73920 non-null float64
finishedsquarefeet13            42 non-null float64
finishedsquarefeet15            3027 non-null float64
finishedsquarefeet50          